In [ ]:
import xarray as xr
import fsspec
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dask.distributed import Client, progress
from pathlib import Path 

In [ ]:
client = Client()
client

In [ ]:
url = 's3://noaa-nwm-retrospective-2-1-zarr-pds/precip.zarr'

In [ ]:
%%time
ds = xr.open_zarr(fsspec.get_mapper(url, anon=True), consolidated=True)

In [ ]:
ds

In [ ]:
## input the coordinates {y1:x1, y2:x2} or {lat1:long1, lat2:long2} format

Coordinates = {3252500:277499, 3253500:277499, 3254500:277499}

In [ ]:

t=0
header = ''
for k, v in Coordinates.items():
    cord = str(k) + ',' + str(v)
    if (t==0):
        a = ds.RAINRATE.sel(y = k, x= v, method="nearest" )
        df = a.to_pandas()
        df =df.to_frame()
        df.rename(columns={0: cord}, inplace=True)
        
    else:
        a1 = ds.RAINRATE.sel(y = k, x= v, method="nearest" )
        df1 = a1.to_pandas()
        df1 =df1.to_frame()
        df1.rename(columns={0: cord}, inplace=True)
        
        df= pd.merge(df, df1, suffixes=(False, False), left_index=True, right_index=True )
    t+=1
    

In [ ]:
df

In [ ]:
filepath = Path('C:/Users/ahmed/Documents/out2.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  